## Migração do Spark para o BigQuery via Dataproc - Parte 1

*  [Parte 1](01_spark - Laboratorio Google.ipynb): O código Spark original, agora em execução no Dataproc (lift-and-shift)
*  [Parte 2](02_gcs - Laboratorio Google.ipynb): Substitua o HDFS pelo Google Cloud Storage. Isso permite clusters específicos de trabalho. (nativo da nuvem)
*  [Parte 3](03_automate - Laboratorio Google.ipynb): Automatize tudo, para que possamos executar em um cluster específico de trabalho. (otimizado para nuvem)
*  [Parte 4](04_bigquery - Laboratorio Google.ipynb): Carregue CSV no BigQuery, use o BigQuery. (modernizar)
*  [Parte 5](05_functions - Laboratorio Google.ipynb): Usando o Cloud Functions, inicie a análise sempre que houver um novo arquivo no intervalo. (sem servidor)

### Copiar dados para HDFS

O código Spark neste bloco de notas é baseado no [git](https://github.com/dipanjanS/data_science_for_all/blob/master/tds_spark_sql_intro/Working%20with%20SQL%20at%20Scale%20-%20Spark%20SQL%20Tutorial.ipynb) associado a [este post](https://opensource.com/article/19/3/apache-spark-and-dataframes-tutorial) por Dipanjan Sarkar. Estou usando para ilustrar a migração de uma carga de trabalho de análise do Spark para o BigQuery via Dataproc.

Os próprios dados vêm da competição KDD de 1999. Vamos pegar 10% dos dados para usar como ilustração.

In [ ]:
!wget http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz

In [ ]:
!hadoop fs -put kddcup* /

In [ ]:
!hadoop fs -ls /

### Leitura de dados

Os dados são arquivos CSV. No Spark, eles podem ser lidos usando Arquivo de texto e dividindo as linhas por vírgulas.

In [ ]:
from pyspark.sql import SparkSession, SQLContext, Row

spark = SparkSession.builder.appName("kdd").getOrCreate()
sc = spark.sparkContext
data_file = "hdfs:///kddcup.data_10_percent.gz"
raw_rdd = sc.textFile(data_file).cache()
raw_rdd.take(5)

In [ ]:
csv_rdd = raw_rdd.map(lambda row: row.split(","))
parsed_rdd = csv_rdd.map(lambda r: Row(
    duration=int(r[0]), 
    protocol_type=r[1],
    service=r[2],
    flag=r[3],
    src_bytes=int(r[4]),
    dst_bytes=int(r[5]),
    wrong_fragment=int(r[7]),
    urgent=int(r[8]),
    hot=int(r[9]),
    num_failed_logins=int(r[10]),
    num_compromised=int(r[12]),
    su_attempted=r[14],
    num_root=int(r[15]),
    num_file_creations=int(r[16]),
    label=r[-1]
    )
)
parsed_rdd.take(5)

### Análise Spark 

Uma maneira de analisar dados no Spark é chamar métodos em um dataframe.

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.createDataFrame(parsed_rdd)
connections_by_protocol = df.groupBy('protocol_type').count().orderBy('count', ascending=False)
connections_by_protocol.show()

Outra maneira é usar o Spark SQL

In [ ]:
df.registerTempTable("connections")
attack_stats = sqlContext.sql("""
                           SELECT 
                             protocol_type, 
                             CASE label
                               WHEN 'normal.' THEN 'no attack'
                               ELSE 'attack'
                             END AS state,
                             COUNT(*) as total_freq,
                             ROUND(AVG(src_bytes), 2) as mean_src_bytes,
                             ROUND(AVG(dst_bytes), 2) as mean_dst_bytes,
                             ROUND(AVG(duration), 2) as mean_duration,
                             SUM(num_failed_logins) as total_failed_logins,
                             SUM(num_compromised) as total_compromised,
                             SUM(num_file_creations) as total_file_creations,
                             SUM(su_attempted) as total_root_attempts,
                             SUM(num_root) as total_root_acceses
                           FROM connections
                           GROUP BY protocol_type, state
                           ORDER BY 3 DESC
                           """)
attack_stats.show()

In [ ]:
%matplotlib inline
ax = attack_stats.toPandas().plot.bar(x='protocol_type', subplots=True, figsize=(10,25))

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.